# Purifying selection - is this reduced on 995F haplotypes? 
## calculate PiN/PiS for 995F, 995S and wt haplotypes
- If we do this per haplotype and take mean, we can get CIs

In [1]:
%run setup.ipynb
import hapclust

In [2]:
# load data
callset_haps = np.load('../data/haps_phase2.npz')
haps = allel.HaplotypeArray(callset_haps['haplotypes'])
pos = allel.SortedIndex(callset_haps['POS'])
ann = callset_haps['ANN']

In [3]:
# chop into gene
start = 2358158
stop = 2431617
loc_vgsc = pos.locate_range(start, stop)
h_vgsc = haps[loc_vgsc]
pos_vgsc = pos[loc_vgsc]
ann_vgsc = ann['Annotation'][loc_vgsc]
h_vgsc

<HaplotypeArray shape=(2092, 2284) dtype=int8>
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
...
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0

In [16]:
#chop into F, S and wt haplotypes
pos_995S = 2422651
pos_995F = 2422652
loc_995S = h_vgsc[pos_vgsc.locate_key(pos_995S)] == 1
loc_995F = h_vgsc[pos_vgsc.locate_key(pos_995F)] == 1
loc_res = loc_995F + loc_995S
loc_wt = ~loc_res

In [17]:
h_vgsc_995F = h_vgsc.compress(loc_995F, axis=1)
h_vgsc_995S = h_vgsc.compress(loc_995S, axis=1)
h_vgsc_wt = h_vgsc.compress(loc_wt, axis=1)

In [18]:
h_vgsc_995F

<HaplotypeArray shape=(2092, 1131) dtype=int8>
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
...
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0

In [19]:
#count alleles
acS = h_vgsc_995S.count_alleles()
acF = h_vgsc_995F.count_alleles()
acw = h_vgsc_wt.count_alleles()
#quick look at first ALT allele
sum(acS[:,1]), sum(acF[:,1]), sum(acw[:,1])

(80372, 183593, 107033)

In [20]:
#get N and S positions
miss_bool = np.asarray([v  == b'missense_variant' for v in ann_vgsc])
pos_vgsc_Non = pos_vgsc.compress(miss_bool)
pos_vgsc_Syn = pos_vgsc.compress(~miss_bool)

In [21]:
#getting weird allel pi results, there is a note in the docs- possibly due to problems working out how many positions it should use
#try chopping ac arrays to N and S first, so that is all it has to work with.

In [28]:
acF_Non = acF.compress(miss_bool, axis=0)
acF_Syn = acF.compress(~miss_bool, axis=0)
piFN = allel.sequence_diversity(pos_vgsc_Non, acF_Non)
piFS = allel.sequence_diversity(pos_vgsc_Syn, acF_Syn)
print(f'995F PiN/PiS = {piFN/piFS}')

995F PiN/PiS = 0.06363847066110961


In [27]:
acS_Non = acS.compress(miss_bool, axis=0)
acS_Syn = acS.compress(~miss_bool, axis=0)
piSN = allel.sequence_diversity(pos_vgsc_Non, acS_Non)
piSS = allel.sequence_diversity(pos_vgsc_Syn, acS_Syn)
print(f'995S PiN/PiS = {piSN/piSS}')

995S PiN/PiS = 0.0015844261633296837


In [26]:
acw_Non = acw.compress(miss_bool, axis=0)
acw_Syn = acw.compress(~miss_bool, axis=0)
piwN = allel.sequence_diversity(pos_vgsc_Non, acw_Non)
piwS = allel.sequence_diversity(pos_vgsc_Syn, acw_Syn)
print(f'wt PiN/PiS = {piwN/piwS}')

wt PiN/PiS = 0.0031758391469860654


# what about dn/ds?